In [ ]:
# Cell 1: imports & paths

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from ultralytics import YOLO
from IPython.display import Video

# Base paths
BASE_DIR = "34759_final_project_rect"
SEQ_DIR  = os.path.join(BASE_DIR, "seq_03")

LEFT_DIR  = os.path.join(SEQ_DIR, "image_02", "data")  # LEFT
RIGHT_DIR = os.path.join(SEQ_DIR, "image_03", "data")  # RIGHT

CALIB_PATH = os.path.join(BASE_DIR, "calib_cam_to_cam.txt")

print("Left images dir :", LEFT_DIR)
print("Right images dir:", RIGHT_DIR)
print("Calib file      :", CALIB_PATH)


In [ ]:
# Cell 2: load calibration (P_rect_02, P_rect_03) and derive intrinsics + baseline

def parse_kitti_calib_cam_to_cam(calib_path):
    """
    Load calib_cam_to_cam.txt and return a dict mapping key -> numpy array.
    For keys with 12 or 9 numbers, reshape to (3,4) or (3,3) respectively.
    """
    params = {}
    with open(calib_path, "r") as f:
        for line in f:
            line = line.strip()
            if len(line) == 0 or line.startswith("#"):
                continue
            if ":" not in line:
                continue
            key, value = line.split(":", 1)
            key = key.strip()
            value = value.strip()
            if len(value) == 0:
                continue
            parts = [float(x) for x in value.split()]
            if len(parts) == 12:
                params[key] = np.array(parts).reshape(3, 4)
            elif len(parts) == 9:
                params[key] = np.array(parts).reshape(3, 3)
            elif len(parts) == 2:
                params[key] = np.array(parts)
            else:
                params[key] = np.array(parts)
    return params

calib = parse_kitti_calib_cam_to_cam(CALIB_PATH)

P2 = calib["P_rect_02"]  # Left camera projection matrix (3x4)
P3 = calib["P_rect_03"]  # Right camera projection matrix (3x4)

print("P_rect_02:\n", P2)
print("\nP_rect_03:\n", P3)

# Intrinsics from left camera
fx = P2[0, 0]
fy = P2[1, 1]
cx = P2[0, 2]
cy = P2[1, 2]

# Baseline: for rectified pair, P3[0,3] = -fx * B (left Tx assumed 0)
baseline = -P3[0, 3] / fx

print(f"\nfx={fx:.3f}, fy={fy:.3f}, cx={cx:.3f}, cy={cy:.3f}, baseline={baseline:.4f} m")


In [ ]:
# Cell 3: list images and helper to load frames

def sorted_image_paths(img_dir):
    files = [f for f in os.listdir(img_dir) if f.lower().endswith(".png")]
    files = sorted(files)
    return [os.path.join(img_dir, f) for f in files]

left_paths  = sorted_image_paths(LEFT_DIR)
right_paths = sorted_image_paths(RIGHT_DIR)

num_frames = min(len(left_paths), len(right_paths))
print(f"Found {num_frames} frames in seq_03")

def load_frame(frame_idx):
    if frame_idx < 0 or frame_idx >= num_frames:
        raise IndexError(f"Frame {frame_idx} out of range [0, {num_frames-1}]")
    left  = cv2.imread(left_paths[frame_idx])
    right = cv2.imread(right_paths[frame_idx])
    return left, right


In [ ]:
# Cell 4: create stereo matcher for disparity

# Parameters can be tuned
window_size = 5
min_disp = 0
num_disp = 128  # must be divisible by 16

stereo = cv2.StereoSGBM_create(
    minDisparity=min_disp,
    numDisparities=num_disp,
    blockSize=7,
    P1=8 * 3 * window_size**2,
    P2=32 * 3 * window_size**2,
    disp12MaxDiff=1,
    uniquenessRatio=10,
    speckleWindowSize=100,
    speckleRange=32
)

def compute_disparity(left_bgr, right_bgr):
    left_gray  = cv2.cvtColor(left_bgr, cv2.COLOR_BGR2GRAY)
    right_gray = cv2.cvtColor(right_bgr, cv2.COLOR_BGR2GRAY)
    disp = stereo.compute(left_gray, right_gray).astype(np.float32) / 16.0
    # Invalid disparities are usually <= 0
    return disp


In [ ]:
# Cell 5: load YOLO detector (pretrained on COCO)

# You can switch to a bigger model if GPU is available, e.g. "yolov8m.pt"
yolo_model = YOLO("yolov8m.pt")

# Map COCO classes to your desired classes
COCO_TO_OUR = {
    "person": "Pedestrian",
    "bicycle": "Bike",
    "car": "Car",
}

def detect_objects(left_bgr, conf_thresh=0.25):
    """
    Run YOLO on left image, return list of detections:
    each detection is dict with keys:
    'type', 'bbox', 'score'
    bbox = [x1, y1, x2, y2] in pixel coordinates.
    """
    results = yolo_model(left_bgr, imgsz=960, verbose=False)[0]

    detections = []
    for box in results.boxes:
        cls_id = int(box.cls[0].item())
        conf   = float(box.conf[0].item())
        if conf < conf_thresh:
            continue
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        coco_name = results.names[cls_id]
        if coco_name not in COCO_TO_OUR:
            continue

        our_type = COCO_TO_OUR[coco_name]
        detections.append({
            "type": our_type,
            "bbox": [x1, y1, x2, y2],
            "score": conf
        })
    return detections


In [ ]:
# Cell 6: 3D estimation helpers

def estimate_box_3d(bbox, disparity_map, fx, fy, cx, cy, baseline, min_valid_pixels=50):
    """
    Given bbox [x1, y1, x2, y2] and disparity map, estimate:
    X, Y, Z (m) and width, height (m) of object in camera coordinates.
    """
    x1, y1, x2, y2 = bbox
    x1_i, y1_i = int(max(0, x1)), int(max(0, y1))
    x2_i, y2_i = int(min(disparity_map.shape[1]-1, x2)), int(min(disparity_map.shape[0]-1, y2))

    if x2_i <= x1_i or y2_i <= y1_i:
        return None  # invalid box

    disp_crop = disparity_map[y1_i:y2_i, x1_i:x2_i]
    valid_mask = disp_crop > 0.5  # threshold to filter garbage disparities

    if valid_mask.sum() < min_valid_pixels:
        return None  # too few valid disparity pixels

    disp_valid = disp_crop[valid_mask]
    median_disp = np.median(disp_valid)

    if median_disp <= 0:
        return None

    # Depth
    Z = fx * baseline / median_disp  # meters

    # Box center in pixels
    u = 0.5 * (x1 + x2)
    v = 0.5 * (y1 + y2)

    # Back-project to 3D
    X = (u - cx) * Z / fx
    Y = (v - cy) * Z / fy

    # Approximate dimensions from pixel size at that depth
    w_px = x2 - x1
    h_px = y2 - y1

    width_m  = w_px * Z / fx
    height_m = h_px * Z / fy

    return {
        "X": float(X),
        "Y": float(Y),
        "Z": float(Z),
        "width_m": float(width_m),
        "height_m": float(height_m),
        "median_disp": float(median_disp),
    }


In [ ]:
# Cell 7: drawing function

def draw_detections(left_bgr, detections_with_3d):
    """
    detections_with_3d: list of dicts with keys:
      'type', 'bbox', 'score', 'X','Y','Z','width_m','height_m' (some may be None if 3D failed)
    """
    img = left_bgr.copy()

    for det in detections_with_3d:
        x1, y1, x2, y2 = det["bbox"]
        obj_type = det["type"]
        score    = det["score"]
        pt1 = (int(x1), int(y1))
        pt2 = (int(x2), int(y2))

        color = (0, 255, 0)  # you can vary by class if you like
        cv2.rectangle(img, pt1, pt2, color, 2)

        # Prepare text lines
        line1 = f"{obj_type} ({score:.2f})"
        if det.get("Z") is not None:
            X, Y, Z = det["X"], det["Y"], det["Z"]
            Wm, Hm  = det["width_m"], det["height_m"]
            line2 = f"Z={Z:.1f}m, X={X:.1f}, Y={Y:.1f}"
            line3 = f"H={Hm:.1f}m, W={Wm:.1f}m"
        else:
            line2 = "3D: n/a"
            line3 = ""

        # Place text above/inside box
        y0 = max(int(y1) - 30, 0)
        cv2.putText(img, line1, (int(x1), y0),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, cv2.LINE_AA)

        y1_txt = max(int(y1) - 12, 0)
        cv2.putText(img, line2, (int(x1), y1_txt),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, cv2.LINE_AA)

        if line3:
            y2_txt = min(int(y2) + 15, img.shape[0]-5)
            cv2.putText(img, line3, (int(x1), y2_txt),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, cv2.LINE_AA)

    return img


In [ ]:
# Cell 8: sample a few frames and visualize detections + 3D info

sample_indices = [0, num_frames // 4, num_frames // 2]  # pick 3 frames across sequence
sample_indices = [idx for idx in sample_indices if 0 <= idx < num_frames]

plt.figure(figsize=(15, 5 * len(sample_indices)))

for i, idx in enumerate(sample_indices):
    left, right = load_frame(idx)
    disparity = compute_disparity(left, right)

    # Detect objects
    dets = detect_objects(left, conf_thresh=0.4)

    dets_with_3d = []
    for det in dets:
        info_3d = estimate_box_3d(det["bbox"], disparity, fx, fy, cx, cy, baseline)
        if info_3d is not None:
            det_with_3d = {**det, **info_3d}
        else:
            det_with_3d = {**det, "X": None, "Y": None, "Z": None,
                           "width_m": None, "height_m": None}
        dets_with_3d.append(det_with_3d)

    img_annotated = draw_detections(left, dets_with_3d)
    img_rgb = cv2.cvtColor(img_annotated, cv2.COLOR_BGR2RGB)

    plt.subplot(len(sample_indices), 1, i + 1)
    plt.imshow(img_rgb)
    plt.title(f"Frame {idx}: {len(dets_with_3d)} detections")
    plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
# Cell 9: process entire sequence and save annotated video

output_video_path = os.path.join("seq_03_annotated_yolo_3d.mp4")
fps = 10  # adjust to taste

# Get frame size from first frame
first_left, _ = load_frame(0)
h, w = first_left.shape[:2]

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(output_video_path, fourcc, fps, (w, h))

for idx in tqdm(range(num_frames), desc="Processing seq_03"):
    left, right = load_frame(idx)
    disparity = compute_disparity(left, right)

    dets = detect_objects(left, conf_thresh=0.4)

    dets_with_3d = []
    for det in dets:
        info_3d = estimate_box_3d(det["bbox"], disparity, fx, fy, cx, cy, baseline)
        if info_3d is not None:
            det_with_3d = {**det, **info_3d}
        else:
            det_with_3d = {**det, "X": None, "Y": None, "Z": None,
                           "width_m": None, "height_m": None}
        dets_with_3d.append(det_with_3d)

    frame_annotated = draw_detections(left, dets_with_3d)
    writer.write(frame_annotated)

writer.release()
print("Saved annotated video to:", output_video_path)


In [ ]:
# Cell 12: 3D Kalman filter and Track class

import numpy as np
from scipy.optimize import linear_sum_assignment

class KalmanFilter3D:
    def __init__(self, dt=0.1, process_var=1.0, meas_var=1.0):
        """
        Simple constant-velocity Kalman filter in 3D.
        State: [X, Y, Z, Vx, Vy, Vz]^T
        Measurement: [X, Y, Z]
        """
        self.dt = dt

        # State transition
        self.F = np.eye(6)
        self.F[0, 3] = dt
        self.F[1, 4] = dt
        self.F[2, 5] = dt

        # Measurement matrix
        self.H = np.zeros((3, 6))
        self.H[0, 0] = 1.0
        self.H[1, 1] = 1.0
        self.H[2, 2] = 1.0

        # Process and measurement noise
        self.Q = process_var * np.eye(6)
        self.R = meas_var * np.eye(3)

    def init_state(self, X, Y, Z):
        x = np.zeros((6, 1))
        x[0, 0] = X
        x[1, 0] = Y
        x[2, 0] = Z
        P = np.eye(6) * 10.0  # large initial uncertainty
        return x, P

    def predict(self, x, P):
        x = self.F @ x
        P = self.F @ P @ self.F.T + self.Q
        return x, P

    def update(self, x, P, meas):
        z = np.array(meas, dtype=float).reshape(3, 1)
        y = z - self.H @ x                       # innovation
        S = self.H @ P @ self.H.T + self.R       # innovation covariance
        K = P @ self.H.T @ np.linalg.inv(S)      # Kalman gain
        x = x + K @ y
        P = (np.eye(6) - K @ self.H) @ P
        return x, P


class Track:
    _next_id = 0

    def __init__(self, det, kf: KalmanFilter3D):
        """
        det: dict with keys 'X','Y','Z','bbox','type','score'
        kf : shared KalmanFilter3D instance
        """
        self.id = Track._next_id
        Track._next_id += 1

        self.kf = kf
        X, Y, Z = det["X"], det["Y"], det["Z"]
        self.x, self.P = self.kf.init_state(X, Y, Z)

        self.bbox = det["bbox"]
        self.type = det["type"]
        self.score = det["score"]

        # Store box size in pixels for reprojection
        x1, y1, x2, y2 = self.bbox
        self.width_px  = float(x2 - x1)
        self.height_px = float(y2 - y1)

        self.age = 1
        self.time_since_update = 0
        self.hits = 1

    def predict(self):
        # Pure motion prediction in 3D
        self.x, self.P = self.kf.predict(self.x, self.P)
        self.age += 1
        self.time_since_update += 1
        # We do NOT update bbox here anymore; we will reproject from 3D state when drawing

    def update(self, det):
        X, Y, Z = det["X"], det["Y"], det["Z"]
        self.x, self.P = self.kf.update(self.x, self.P, [X, Y, Z])
        self.bbox = det["bbox"]
        self.type = det["type"]
        self.score = det["score"]

        # Update box size in pixels from measurement
        x1, y1, x2, y2 = self.bbox
        self.width_px  = float(x2 - x1)
        self.height_px = float(y2 - y1)

        self.time_since_update = 0
        self.hits += 1

    def get_state(self):
        X = float(self.x[0, 0])
        Y = float(self.x[1, 0])
        Z = float(self.x[2, 0])
        return X, Y, Z


In [ ]:
# Cell 13: associate detections with existing tracks using 3D distance

def associate_detections_to_tracks(tracks, detections, max_dist=5.0):
    """
    tracks:    list[Track]
    detections: list[det dict with 'X','Y','Z']
    Returns:
        matches: list of (track_idx, det_idx)
        unmatched_tracks: list of track indices
        unmatched_detections: list of detection indices
    """
    if len(tracks) == 0:
        return [], [], list(range(len(detections)))
    if len(detections) == 0:
        return [], list(range(len(tracks))), []

    cost = np.zeros((len(tracks), len(detections)), dtype=float)

    for ti, tr in enumerate(tracks):
        X_t, Y_t, Z_t = tr.get_state()
        for di, det in enumerate(detections):
            X_d, Y_d, Z_d = det["X"], det["Y"], det["Z"]
            if X_d is None or Y_d is None or Z_d is None:
                cost[ti, di] = 1e6  # basically "infinite"
            else:
                dist = np.linalg.norm([X_t - X_d, Y_t - Y_d, Z_t - Z_d])
                cost[ti, di] = dist

    row_ind, col_ind = linear_sum_assignment(cost)

    matches = []
    unmatched_tracks = list(range(len(tracks)))
    unmatched_detections = list(range(len(detections)))

    for r, c in zip(row_ind, col_ind):
        if cost[r, c] > max_dist:
            continue  # too far, don't match
        matches.append((r, c))
        if r in unmatched_tracks:
            unmatched_tracks.remove(r)
        if c in unmatched_detections:
            unmatched_detections.remove(c)

    return matches, unmatched_tracks, unmatched_detections


In [ ]:
# Cell 14: draw tracks (with IDs, 3D positions, and occlusion handling)

def draw_tracks(left_bgr, tracks, fx, fy, cx, cy):
    """
    Draws all active tracks on the image.
    For tracks with time_since_update > 0, we draw predicted-only positions (yellow).
    For updated tracks, we draw in green.
    """
    img = left_bgr.copy()
    H, W = img.shape[:2]

    for tr in tracks:
        X, Y, Z = tr.get_state()

        # If depth goes crazy, skip drawing
        if Z <= 0.5 or Z > 200.0:
            continue

        # Project 3D center -> image coordinates
        u = fx * X / Z + cx
        v = fy * Y / Z + cy

        # Use stored box size (in pixels) to build bbox around (u, v)
        w = tr.width_px
        h = tr.height_px

        x1 = int(u - w / 2)
        y1 = int(v - h / 2)
        x2 = int(u + w / 2)
        y2 = int(v + h / 2)

        # Clamp to image bounds
        x1 = max(0, min(W - 1, x1))
        x2 = max(0, min(W - 1, x2))
        y1 = max(0, min(H - 1, y1))
        y2 = max(0, min(H - 1, y2))

        # Color: green if updated this frame, yellow if only predicted
        if tr.time_since_update == 0:
            color = (0, 255, 0)   # updated
            status = ""
        else:
            color = (0, 255, 255) # predicted
            status = " (pred)"

        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)

        line1 = f"ID {tr.id} {tr.type}{status}"
        line2 = f"Z={Z:.1f}m, X={X:.1f}, Y={Y:.1f}"

        y0 = max(y1 - 25, 0)
        y1_txt = max(y1 - 7, 0)

        cv2.putText(img, line1, (x1, y0),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, cv2.LINE_AA)
        cv2.putText(img, line2, (x1, y1_txt),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, cv2.LINE_AA)

    return img


In [ ]:
# Cell 15: run tracker on a few consecutive frames for visualization

# Make sure fps exists; if not, define it (used for dt)
try:
    fps
except NameError:
    fps = 10.0

tracks = []
Track._next_id = 0

kf_3d = KalmanFilter3D(dt=1.0 / fps, process_var=5.0, meas_var=1.0)
max_age = 50        # frames to keep "dead" tracks alive (3s if fps=10)
max_dist = 40.0     # max 3D distance in meters for association
min_3d_conf_pixels = 50  # you can keep this, or slightly lower if needed

start_idx = 0
end_idx = min(start_idx + 20, num_frames)  # small window

import matplotlib.pyplot as plt

plt.figure(figsize=(15, 4 * (end_idx - start_idx)//5 + 5))

plot_row = 0

for frame_idx in range(start_idx, end_idx):
    left, right = load_frame(frame_idx)
    disparity = compute_disparity(left, right)

    # Step 1: detect objects in this frame
    dets = detect_objects(left, conf_thresh=0.4)

    # Step 2: get 3D for each detection (skip those without valid 3D)
    dets3d = []
    for det in dets:
        info_3d = estimate_box_3d(det["bbox"], disparity, fx, fy, cx, cy, baseline,
                                  min_valid_pixels=min_3d_conf_pixels)
        if info_3d is not None:
            dets3d.append({**det, **info_3d})

    # Step 3: predict all existing tracks
    for tr in tracks:
        tr.predict()

    # Step 4: associate detections to tracks
    matches, unmatched_tracks, unmatched_dets = associate_detections_to_tracks(
        tracks, dets3d, max_dist=max_dist
    )

    # Step 5: update matched tracks
    for t_idx, d_idx in matches:
        tracks[t_idx].update(dets3d[d_idx])

    # Step 6: create new tracks for unmatched detections
    for d_idx in unmatched_dets:
        tr = Track(dets3d[d_idx], kf_3d)
        tracks.append(tr)

    # Step 7: remove dead tracks
    new_tracks = []
    for tr in tracks:
        if tr.time_since_update <= max_age:
            new_tracks.append(tr)
    tracks = new_tracks

    # Visualize every 5th frame
    if frame_idx % 5 == 0:
        plot_row += 1
        img_tr = draw_tracks(left, tracks, fx, fy, cx, cy)
        img_tr_rgb = cv2.cvtColor(img_tr, cv2.COLOR_BGR2RGB)

        plt.subplot((end_idx - start_idx) // 5 + 1, 1, plot_row)
        plt.imshow(img_tr_rgb)
        plt.title(f"Frame {frame_idx} with {len(tracks)} active tracks")
        plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
# Cell 16: full seq_03 tracking video with IDs and occlusion prediction

output_video_path_tracks = os.path.join("seq_03_tracking_3d.mp4")
fps = 10  # same as before, or adjust

first_left, _ = load_frame(0)
h, w = first_left.shape[:2]

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(output_video_path_tracks, fourcc, fps, (w, h))

# Re-init tracker state for full run
kf_3d = KalmanFilter3D(dt=1.0 / fps, process_var=5.0, meas_var=1.0)
max_age = 50        # frames to keep "dead" tracks alive (3s if fps=10)
max_dist = 40.0     # max 3D distance in meters for association
min_3d_conf_pixels = 50  # you can keep this, or slightly lower if needed

from tqdm import tqdm

for frame_idx in tqdm(range(num_frames), desc="Tracking seq_03"):
    left, right = load_frame(frame_idx)
    disparity = compute_disparity(left, right)

    dets = detect_objects(left, conf_thresh=0.4)

    dets3d = []
    for det in dets:
        info_3d = estimate_box_3d(det["bbox"], disparity, fx, fy, cx, cy, baseline,
                                  min_valid_pixels=min_3d_conf_pixels)
        if info_3d is not None:
            dets3d.append({**det, **info_3d})

    # Predict step
    for tr in tracks:
        tr.predict()

    # Associate
    matches, unmatched_tracks, unmatched_dets = associate_detections_to_tracks(
        tracks, dets3d, max_dist=max_dist
    )

    # Update matched
    for t_idx, d_idx in matches:
        tracks[t_idx].update(dets3d[d_idx])

    # New tracks
    for d_idx in unmatched_dets:
        tracks.append(Track(dets3d[d_idx], kf_3d))

    # Remove stale
    new_tracks = []
    for tr in tracks:
        if tr.time_since_update <= max_age:
            new_tracks.append(tr)
    tracks = new_tracks

    # Draw
    frame_annotated = draw_tracks(left, tracks, fx, fy, cx, cy)
    writer.write(frame_annotated)

writer.release()
print("Saved tracking video to:", output_video_path_tracks)
